
# 作業目標:
    
    了解keras 模型架構


# 作業重點:
    
    請修改input shape: (Conv2D(64, (3, 3))的設定, 新增一層 Dense 並觀看 model.summary 的輸出


In [1]:
# 載入必須使用的 Library
import keras
from keras.datasets import cifar10
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 10

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# check shape 以符合CNN的 input是四維矩陣 需求，因為 RGB圖片的格式為 ID數維度, width, height, channels(RGB=3,Gray=1)
print('x_train shape:', x_train.shape) 
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# Onehot encoding，將彼此間不相關且非有序的categories轉換為連續性且是有序數值
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# build our CNN model

# 建立一個線性堆疊模型，之後只要將建立的神經網路層依次加入即可
model = Sequential()

#第一個卷積層並加入model，該層有64個 filters（即我們稱的 filter或 kernel），kernel大小為3×3，
#卷積後的圖形大小不變（即圖像周圍會補值），輸入的圖形維度為32x32x3，使用 relu啟動函數。
#由於定義了64個 filters，因此本層會輸出 64個 32×32的影像。
#model.add(Conv2D(filters=64, kernel_size=(3, 3), padding=’same’, input_shape=(32, 32, 3), activation=’relu’))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

#加入池化層，本層參數比較少，只要定義pool size為(2,2)，即尺寸縮小為一半。
#本層接收了上一層卷積層的64個32×32影像後，會輸出16個16×16的影像
model.add(MaxPooling2D(pool_size=(2, 2)))

#建立第二個卷積層，定義該層有36個filters，kernel大小為5×5，卷積後的圖形大小不變（即圖像周圍會補值），
#輸入的圖形維度為(16,16,3)但可省略因為Keras,會自動判斷，使用relu啟動函數。
#由於定義了36個filters，因此本層會輸出36個16×16的影像。
model.add(Conv2D(36, (5, 5), padding='same', input_shape=(16, 16, 3), activation='relu') )

#加入第二個池化層，參數與第一個池化層完全相同。本層會接收36個16×16影像後，輸出36個 8×8的影像。
model.add(MaxPooling2D(pool_size=(2, 2)))

#平坦層，將特徵值轉為一維資料以供後續的全連結層使用。本層接收第二個池化層傳來的36x8x8（=1764）資料之後，轉為1維的2304。
model.add(Flatten())

#建立全連結層中的隱藏層(即傳統神經網路中的全連結層)，指定其神經元數目為512個（此數目可調整），啟用函數使用Relu。
#model.add(Dense(512, activation=’relu’))
model.add(Dense(512))
model.add(Activation('relu'))


#加入Dropout層，它的功用是減少過度擬合。
#在深度學習的訓練過程中，Dropout會讓每次batch run都依據機率丟棄一定比例的神經元不予計算，使得每一次都好像在訓練不同的神經網路一樣。
#下方的程式中，定義該Dropout層每次訓練時要丟棄50%的神經元。
model.add(Dropout(0.5))

#本模型的最後一層是輸出層，也就是要輸出 (num_classes = 10)十種 0~9的分類值，一般使用softmax作為分類模型輸出層的啟動函數。
#model.add(Dense(num_classes))
#model.add(Activation('softmax'))
model.add(Dense(num_classes, activation='softmax'))



W0702 17:02:07.331780 26356 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0702 17:02:07.344207 26356 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0702 17:02:07.345696 26356 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0702 17:02:07.355092 26356 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0702 17:02:07.378435 26356 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py

In [4]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 36)        57636     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1180160   
__________